# YouTube Clipper

<table class="tfo-notebook-buttons" align="left">

  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1cnvPna6JC4nO0vm0Eu5dm4cx2FoRm-kQ?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Executar no Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/eduardoboca/YouTubeClipper/blob/main/src/YoutubeClipper.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Ver código no GitHub</a>
  </td>
</table>

Este projeto amplia as capacidades do Gemini para extrair informações sobre vídeos, fornecendo uma solução adaptável para análise de conteúdo de vídeo em larga escala.

Ele se integra ao YouTube para processar vídeos de qualquer tamanho, mitigando as limitações do Gemini quanto ao tamanho do vídeo e download de vídeos do YouTube.

A aplicação consiste em quatro partes principais:
*   Configurações iniciais: instalação das bibliotecas necessárias e configurações de debug da aplicação.
*   Integração com a API do YouTube para efetuar o download do vídeo e extrair o transcript.
*   Integração com o Gemini para resumir automaticamente as transcrições dos vídeos processados.
*   Exibição do Resultado: Exibição dos resultados processados, incluindo o resumo e a timeline do vídeo.





## Configurações Iniciais

### Modo de Depuração (Debug Mode)

A função `debug_print` foi criada para imprimir mensagens de depuração apenas quando o modo de depuração está ativado. Para ativar o modo de depuração, defina a variável `debug` como `True`.


In [ ]:
debug = False

def debug_print(message):
    if debug:
        print(message)

### Passo 1: instalar o SDK do Gemini e as apis de download e transcrição do YouTube

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
!pip install -q youtube-transcript-api


In [ ]:
!pip install -q pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00


## Integração com o YouTube

### Passo 2: fazer o download e a transcrição do vídeo no YouTube



Como exemplo, vamos utilizar o vídeo da aula 4 da imersão em Gemini da Alura

Mas você pode testar com qualquer vídeo! Basta modificar o valor da variável *video_url* no código abaixo com o endereço do seu vídeo

In [ ]:
#  URL do vídeo que você quer baixar
video_url = "https://www.youtube.com/watch?v=iwt4bOIHy7s"

from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
import json

try:
    yt = YouTube(video_url)
    video_stream = yt.streams.get_highest_resolution()

    # Download do vídeo
    video_stream.download()

    # Extrair o ID do vídeo ID from the URL
    video_id = video_url.split("v=")[1]

    # Transcript do vídeo
    transcripts = YouTubeTranscriptApi.get_transcript(video_id, languages=['pt', 'en'])

    if transcripts:
        subtitle_text = " ".join([entry["text"] for entry in transcripts])

        debug_print("*** Transcripts:")
        debug_print(subtitle_text)
    else:
        debug_print("Transcript não disponível para este vídeo.")

except Exception as e:
    debug_print(f"Erro: {str(e)}")




### Passo 3: dividir o transcript em pedaços menores para poder enviar ao Gemini.

In [ ]:
def split_transcript(transcript, max_duration=600):
    parts = []
    current_part = []
    current_duration = 0

    for entry in transcript:
        if current_duration + entry["duration"] <= max_duration:
            # Serializa a entrada para JSON e adiciona à parte atual
            current_part.append(json.dumps(entry))
            current_duration += entry["duration"]
        else:
            # Decodifica cada entrada de transcrição na parte atual
            decoded_part = [bytes(entry, "utf-8").decode("unicode_escape") for entry in current_part]
            parts.append(decoded_part)  # Adiciona a parte atual à lista de partes
            current_part = [json.dumps(entry)]  # Começa uma nova parte com a entrada atual
            current_duration = entry["duration"]

    # Decodifica a última parte
    decoded_part = [bytes(entry, "utf-8").decode("unicode_escape") for entry in current_part]
    parts.append(decoded_part)

    return parts


# Dividir o transcript em partes
segments = split_transcript(transcripts, max_duration=600)

debug_print (f"Transcript dividido em {len(segments)} partes")
for segment in segments:
  debug_print (f"{segment[:1000]}")
  debug_print ("---")


## Integração com o Gemini




### Passo 4: configurar a API Key do Gemini






In [ ]:
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

### Passo 5: configurar o modelo

In [ ]:
generation_config = {
    "candidate_count" : 1,
    "temperature" : 0.5
}

safety_settings = {
    "HARASSMENT" : "BLOCK_NONE",
    "HATE" : "BLOCK_NONE",
    "SEXUAL" : "BLOCK_NONE",
    "DANGEROUS" : "BLOCK_NONE"
}

model = genai.GenerativeModel("gemini-1.0-pro",
                              generation_config = generation_config,
                              safety_settings = safety_settings)

In [ ]:
prompt = "Preciso descrever o conteúdo de um vídeo do YouTube."
prompt += "\n".join(segments[0])

debug_print(prompt)

### Passo 6: solicitar o resumo ao Gemini
Primeiro vamos solicitar o resumo e a timeline de cada trecho


In [ ]:
summaries = []
clips = []

for segment in segments:
  prompt = "Preciso descrever o conteúdo de um vídeo do YouTube."
  prompt += " Aqui está o conteúdo:"
  prompt += "\n".join(segment)
  prompt += " Os dados do vídeo estão no formato JSON {text:'', start:'', duration:''}. start representa o início do trecho em segundos, e duration representa a duração do trecho em segundos;"
  prompt += " Quais são os assuntos discutidos nesse vídeo? Crie uma lista contendo uma descrição do assunto e o valor do campo start em que se inicia o assunto."
  prompt += " Também preciso que você faça um breve resumo do vídeo inteiro."
  prompt += " Responda no formato JSON, de forma que uma aplicação possa fazer o parse da resposta. Os campos do JSON devem ser exatamente estes: 1. summary: resumo do vídeo e 2: topics: lista dos assuntos, que também deve ter um formato JSON assim definido: 1. description e 2.start"
  prompt += " O valor de start deve ser exatamente igual ao encontrado no json original."
  response = model.generate_content(prompt)
  response_text = response.text
  response_text = response_text.replace("```json", '')
  response_text = response_text.replace("```", '')

  json_object = json.loads(response_text)

  summaries.append(json_object["summary"])
  clips.extend(json_object["topics"])

for clip in clips:
    debug_print(clip)


### Passo 7: juntar todos os trechos e montar o resumo geral

In [ ]:
summaries_string = '\n'.join(summaries)

prompt = "Preciso descrever o conteúdo de um vídeo do YouTube."
prompt += " O vídeo contém vários trechos. Vou enviar o resumo de cada trecho. Você deve ler todos eles e gerar um resumo geral."
prompt += " Retorne apenas o resumo geral. Estes são os resumos:\n"

prompt += summaries_string
response = model.generate_content(prompt)

total_summary = response.text


## E finalmente temos o resumo do vídeo e a timeline!

In [ ]:
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

display (to_markdown("### Resumo:"))
display (to_markdown(total_summary))
display (to_markdown("### Timeline:"))

for clip in clips:
  if isinstance(clip['start'], str):
    clip['start'] = clip['start'].replace("'", "")
    clip['start'] = float(clip['start'])

for clip in clips:
    link = f"{video_url}&t={clip['start']}s"
    hours = int(clip['start'] // 3600)
    minutes = int((clip['start'] % 3600) // 60)
    seconds = int(clip['start'] % 60)

    # Formato do tempo HH:MM:SS
    time_formatted = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    #time_formatted = clip['start']
    start_aligned = f"{time_formatted}".ljust(10)
    description_aligned = clip['description'].ljust(50)
    display (to_markdown(f"{start_aligned} - [{description_aligned}]({link})"))



> ### Resumo:

> Este vídeo abrangente fornece uma compreensão aprofundada de embeddings, representações vetoriais de palavras que permitem que os computadores entendam o significado semântico do texto. Ele aborda os conceitos fundamentais de embeddings, suas aplicações em processamento de linguagem natural e a criação de modelos de embeddings usando o Gemini da IA generativa. O vídeo também orienta os espectadores na geração de embeddings de texto, realização de consultas semânticas e criação de soluções de busca de texto usando embeddings e NumPy. Além disso, ele demonstra como combinar embeddings com modelos generativos para gerar respostas personalizadas.

> ### Timeline:

> 00:00:00   - [Introdução à aula e visão geral dos objetivos     ](https://www.youtube.com/watch?v=Ih64Ad5eots&t=0.001s)

> 00:00:36   - [Explicação do conceito de embeddings e sua importância para o acesso a documentos](https://www.youtube.com/watch?v=Ih64Ad5eots&t=36.099s)

...

> 01:02:22   - [Critérios de avaliação do projeto                 ](https://www.youtube.com/watch?v=Ih64Ad5eots&t=3742.466s)

> 01:04:34   - [Dicas para usar o AI Studio e código de exemplo   ](https://www.youtube.com/watch?v=Ih64Ad5eots&t=3874.158s)

> 01:06:17   - [Promoção dos cursos e escolas da Alura            ](https://www.youtube.com/watch?v=Ih64Ad5eots&t=3977.623s)

> 01:07:39   - [Convite para ingressar na comunidade Google Cloud Innovators](https://www.youtube.com/watch?v=Ih64Ad5eots&t=4059.075s)

> 01:08:07   - [Convite para assistir ao Google I/O               ](https://www.youtube.com/watch?v=Ih64Ad5eots&t=4087.15s)

## Extra: Identificar *qualquer palavra ou frase* no vídeo



### Parte 1:
Buscar a a palavra em cada trecho

In [ ]:
debug = False
clips = []
frase = "desafio"

def is_valid_json(json_str):
    try:
        json.loads(json_str)
        return True
    except json.JSONDecodeError:
        return False

for segment in segments:
  prompt = "Preciso buscar uma frase em um vídeo"
  prompt += " Aqui está o transcript do vídeo:"
  prompt += "\n".join(segment)
  prompt += " Os dados do vídeo estão no formato JSON {text:'', start:'', duration:''}. start representa o início do trecho em segundos, e duration representa a duração do trecho em segundos;"
  prompt += f" Identifique os trechos em que a expressão {frase} é falada"
  prompt += " Responda no formato JSON, de forma que uma aplicação possa fazer o parse da resposta. Os campos do JSON devem ser exatamente estes: topics: lista dos assuntos, que também deve ter um formato JSON assim definido: 1. description e 2.start"
  response = model.generate_content(prompt)
  response_text = response.text
  response_text = response_text.replace("```json", '')
  response_text = response_text.replace("```", '')

  if is_valid_json(response_text):
    json_object = json.loads(response_text)
    clips.extend(json_object["topics"])

for clip in clips:
    debug_print(clip)


### Parte 2: formatação e impressão do resultado

In [ ]:
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

display (to_markdown("### Resultado:"))

for clip in clips:
  if isinstance(clip['start'], str):
    clip['start'] = clip['start'].replace("'", "")
    clip['start'] = float(clip['start'])

for clip in clips:
    link = f"{video_url}&t={clip['start']}s"
    hours = int(clip['start'] // 3600)
    minutes = int((clip['start'] % 3600) // 60)
    seconds = int(clip['start'] % 60)

    # Formato do tempo HH:MM:SS
    time_formatted = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    #time_formatted = clip['start']
    start_aligned = f"{time_formatted}".ljust(10)
    description_aligned = clip['description'].ljust(50)
    display (to_markdown(f"{start_aligned} - [{description_aligned}]({link})"))

> ### Resultado:

> 00:20:20   - [IA Alura + Google                                 ](https://www.youtube.com/watch?v=Ih64Ad5eots&t=1220.179s)

> 01:02:08   - [Plano Pro da Alura na faixa,                      ](https://www.youtube.com/watch?v=Ih64Ad5eots&t=3728.002s)

> 01:03:03   - [Você pode fazer quantos projetos você quiser,     ](https://www.youtube.com/watch?v=Ih64Ad5eots&t=3783.862s)

> 01:04:29   - [E também, um outro ponto é que tem que ter relação com o conteúdo da imersão.](https://www.youtube.com/watch?v=Ih64Ad5eots&t=3869.94s)

...

> 01:07:23   - [temos até curso de Excel,                         ](https://www.youtube.com/watch?v=Ih64Ad5eots&t=4043.658s)

> 01:07:25   - [coisas de programação front-end, back-end, gestão e inovação também.](https://www.youtube.com/watch?v=Ih64Ad5eots&t=4045.95s)

> 01:07:29   - [Tem várias escolas na Alura,                      ](https://www.youtube.com/watch?v=Ih64Ad5eots&t=4049.842s)

> 01:07:31   - [na segunda-feira você vai ter a oportunidade de se matricular](https://www.youtube.com/watch?v=Ih64Ad5eots&t=4051.476s)

> 01:07:34   - [com esse descontaço!                              ](https://www.youtube.com/watch?v=Ih64Ad5eots&t=4054.359s)